# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:

vacay_df=pd.read_csv('output_data/clean_city_data.csv').reset_index()
vacay_df.drop(["level_0","Unnamed: 0", "index"], axis=1, inplace=True)
vacay_df.head()

,City,Latitude,Longitude,Max Temperature (Celsius),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Namatanai,-3.6667,152.4333,27.31,81,99,3.14,PG,02/04/21 02:20
1,Bredasdorp,-34.5322,20.0403,22.00,68,13,2.06,ZA,02/04/21 02:16
2,Chara,56.9064,118.2631,-25.50,78,100,0.25,RU,02/04/21 02:20
3,Puerto Ayora,-0.7393,-90.3518,24.44,94,98,3.53,EC,02/04/21 02:20
4,New Norfolk,-42.7826,147.0587,26.67,48,93,1.34,AU,02/04/21 02:20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [13]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacay_df[["Latitude", "Longitude"]].astype(float)
humidity=vacay_df["Humidity (%)"].astype(float)
locations

,Latitude,Longitude
0,-3.6667,152.4333
1,-34.5322,20.0403
2,56.9064,118.2631
3,-0.7393,-90.3518
4,-42.7826,147.0587
...,...,...
559,59.7000,150.1667
560,-33.0810,-68.4681
561,-5.5569,-80.8222
562,21.2833,-88.2667


In [37]:
#Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

In [38]:
#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

In [39]:
#add layer
fig.add_layer(heat_layer)

In [40]:
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [59]:
ideal_weather_df=vacay_df[(vacay_df["Max Temperature (Celsius)"]<=28) & 
                        (vacay_df["Max Temperature (Celsius)"]>=22) & 
                        (vacay_df["Wind Speed (mph)"]<10) & 
                        (vacay_df["Cloudiness (%)"]<=10)&
                        (vacay_df["Humidity (%)"]<=50)].reset_index()
ideal_weather_df.head()

,index,City,Latitude,Longitude,Max Temperature (Celsius),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,7,Tura,25.5198,90.2201,27.04,30,0,1.15,IN,02/04/21 02:20
1,24,Busselton,-33.6500,115.3333,23.89,41,0,1.34,AU,02/04/21 02:20
2,155,Umm Hājar,13.2954,19.6966,22.49,20,0,6.03,TD,02/04/21 02:21
3,191,Pokhara,28.2333,83.9833,23.11,20,0,2.96,NP,02/04/21 02:21
4,202,Hong Kong,22.2855,114.1577,23.00,49,0,3.60,HK,02/04/21 02:18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
#hotel_name=[]
#hotel_df=pd.DataFrame({"Hotel Name":hotel_name})
ideal_weather_df["Hotel Name"]=""
hotel_df=ideal_weather_df
hotel_df.head()

,index,City,Latitude,Longitude,Max Temperature (Celsius),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,7,Tura,25.5198,90.2201,27.04,30,0,1.15,IN,02/04/21 02:20,
1,24,Busselton,-33.6500,115.3333,23.89,41,0,1.34,AU,02/04/21 02:20,
2,155,Umm Hājar,13.2954,19.6966,22.49,20,0,6.03,TD,02/04/21 02:21,
3,191,Pokhara,28.2333,83.9833,23.11,20,0,2.96,NP,02/04/21 02:21,
4,202,Hong Kong,22.2855,114.1577,23.00,49,0,3.60,HK,02/04/21 02:18,


In [63]:
params={"radius":5000,"types":"hotels","key":g_key}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
